In [1]:
#import necessary modules
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd

In [2]:
#create SQLite engine
engine = create_engine("sqlite:///sqlite_db_pythonsqlite.db")

In [3]:
#inspect and look at the tables list, columns
insp = inspect(engine) #you need inspect object to get table names and columns in each table
insp.get_table_names()

['Bookings', 'Facilities', 'Members']

In [4]:
#get columns in each of the tables in the db
#get_columns returns a list of columns in a table as dictionary object with name, type etc
#so we use list comprehension to get just the name 

for table in ['Bookings', 'Facilities', 'Members']: 
    print([item['name'] for item in insp.get_columns(table)])

['bookid', 'facid', 'memid', 'starttime', 'slots']
['facid', 'name', 'membercost', 'guestcost', 'initialoutlay', 'monthlymaintenance']
['memid', 'surname', 'firstname', 'address', 'zipcode', 'telephone', 'recommendedby', 'joindate']


/opt/anaconda3/lib/python3.8/site-packages/sqlalchemy/dialects/sqlite/base.py:2184: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['4']; using no arguments.
  util.warn(
/opt/anaconda3/lib/python3.8/site-packages/sqlalchemy/dialects/sqlite/base.py:2184: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['1']; using no arguments.
  util.warn(
/opt/anaconda3/lib/python3.8/site-packages/sqlalchemy/dialects/sqlite/base.py:2184: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['2']; using no arguments.
  util.warn(
/opt/anaconda3/lib/python3.8/site-packages/sqlalchemy/dialects/sqlite/base.py:2184: SAWarning: Could not instantiate type <class 'sqlalchemy.sql.sqltypes.INTEGER'> with reflected arguments ['5']; using no arguments.
  util.warn(


In [5]:
#use pandas in-built function to fire-up the engine and start querying
pd.read_sql_query("SELECT * from Bookings LIMIT 5", engine)

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [6]:
pd.read_sql_query("SELECT * from Facilities LIMIT 5", engine)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


In [7]:
pd.read_sql_query("SELECT * from Members LIMIT 5", engine)

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


### Q1: Some of the facilities charge a fee to members, but some do not. Write a SQL query to produce a list of the names of the facilities that do.


In [8]:
query_1 = """SELECT name, membercost FROM Facilities WHERE membercost>0;"""
pd.read_sql_query(query_1, engine)

,name,membercost
0,Tennis Court 1,5.0
1,Tennis Court 2,5.0
2,Massage Room 1,9.9
3,Massage Room 2,9.9
4,Squash Court,3.5


### Q2: How many facilities do not charge a fee to members?

In [9]:
query_2 = """SELECT COUNT(*) AS count FROM Facilities WHERE membercost=0;"""
pd.read_sql_query(query_2, engine)

,count
0,4


### Q3: Write an SQL query to show a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost. Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [10]:
query_3 = """SELECT facid, name, membercost, monthlymaintenance 
FROM Facilities 
WHERE membercost<0.2*monthlymaintenance 
AND membercost>0;
"""
pd.read_sql_query(query_3, engine)

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,4,Massage Room 1,9.9,3000
3,5,Massage Room 2,9.9,3000
4,6,Squash Court,3.5,80


### Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5. Try writing the query without using the OR operator.

In [11]:
query_4 = """
SELECT * FROM Facilities WHERE facid IN (1,5);
"""
pd.read_sql_query(query_4, engine)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


### Q5: Produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than \\$100. Return the name and monthly maintenance of the facilities in question.

In [12]:
query_5 = """
SELECT name, monthlymaintenance,
CASE WHEN monthlymaintenance>100 THEN 'expensive'
ELSE 'cheap' END AS economical
FROM Facilities
LIMIT 5;
"""
pd.read_sql_query(query_5, engine)

,name,monthlymaintenance,economical
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive


### Q6: You'd like to get the first and last name of the last member(s) who signed up. Try not to use the LIMIT clause for your solution.

In [13]:
query_6 = """
SELECT firstname, surname, joindate FROM Members ORDER BY joindate DESC LIMIT 5;
"""
pd.read_sql_query(query_6, engine)

,firstname,surname,joindate
0,Darren,Smith,2012-09-26 18:08:45
1,Erica,Crumpet,2012-09-22 08:36:38
2,John,Hunt,2012-09-19 11:32:45
3,Hyacinth,Tupperware,2012-09-18 19:32:05
4,Millicent,Purview,2012-09-18 19:04:01


### Q7: Produce a list of all members who have used a tennis court. Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name.

In [14]:
query_7 = """
SELECT DISTINCT f.name || ' ' || m.firstname
FROM Bookings AS b
JOIN Facilities AS f
ON f.facid = b.facid
JOIN Members AS m
ON b.memid = m.memid
WHERE b.facid IN (0,1)
ORDER BY m.firstname
LIMIT 5;
"""
pd.read_sql_query(query_7, engine)

,f.name || ' ' || m.firstname
0,Tennis Court 1 Anne
1,Tennis Court 2 Anne
2,Tennis Court 2 Burton
3,Tennis Court 1 Burton
4,Tennis Court 1 Charles


### Q8: Produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than \\$30. Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.

In [15]:
query_8 = """
SELECT f.name || ' ' || m.firstname, 
CASE WHEN b.memid = 0 THEN membercost*slots 
ELSE guestcost*slots END AS cost
FROM Bookings AS b
JOIN Facilities AS f
ON b.facid = f.facid
JOIN Members AS m
ON b.memid = m.memid
WHERE starttime LIKE '2012-09-14%' 
GROUP BY firstname
HAVING cost>30
ORDER BY cost DESC
LIMIT 5;
"""
pd.read_sql_query(query_8, engine)

,f.name || ' ' || m.firstname,cost
0,Massage Room 1 Jemima,160
1,Massage Room 1 Jack,160
2,Tennis Court 1 Tim,75
3,Tennis Court 1 Gerald,75
4,Tennis Court 1 Douglas,75


### Q9: This time, produce the same result as in Q8, but using a subquery.

In [16]:
query_9 = """
SELECT * FROM (
SELECT f.name || ' ' || m.firstname, 
CASE WHEN b.memid = 0 THEN membercost*slots 
ELSE guestcost*slots END AS cost
FROM Bookings AS b
JOIN Facilities AS f
ON b.facid = f.facid
JOIN Members AS m
ON b.memid = m.memid
WHERE starttime LIKE '2012-09-14%' 
GROUP BY firstname
ORDER BY cost DESC) AS r
WHERE cost>30
LIMIT 5;
"""
pd.read_sql_query(query_9, engine)

,f.name || ' ' || m.firstname,cost
0,Massage Room 1 Jemima,160
1,Massage Room 1 Jack,160
2,Tennis Court 1 Tim,75
3,Tennis Court 1 Gerald,75
4,Tennis Court 1 Douglas,75


### Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [17]:
query_10 = """SELECT f.name,
            SUM(CASE WHEN m.memid = 0 THEN guestcost*slots
            ELSE membercost*slots END) AS total_revenue
            FROM Facilities AS f
            JOIN Bookings AS b
            ON f.facid = b.facid
            JOIN Members AS m
            ON b.memid = m.memid
            GROUP BY f.name
            HAVING total_revenue < 1000
            ORDER BY total_revenue DESC;
            """
pd.read_sql_query(query_10, engine)

,name,total_revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


### Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [18]:
query_11 = """
SELECT m1.firstname || ' ' || m1.surname AS member, m2.firstname || ' ' || m2.surname AS recommendedby
FROM Members as m1
JOIN Members as m2
ON m1.recommendedby = m2.memid
WHERE m1.recommendedby IS NOT NULL
ORDER BY m1.surname, m1.firstname
LIMIT 5;
"""
pd.read_sql_query(query_11, engine)

,member,recommendedby
0,Florence Bader,Ponder Stibbons
1,Anne Baker,Ponder Stibbons
2,Timothy Baker,Jemima Farrell
3,Tim Boothe,Tim Rownam
4,Gerald Butters,Darren Smith


### Q12: Find the facilities with their usage by member, but not guests 

In [19]:
query_12 = """
SELECT DISTINCT f.name, COUNT(*) AS member_usage
FROM Bookings AS b
JOIN Facilities AS f
ON f.facid = b.facid
JOIN Members AS m
ON m.memid = b.memid
WHERE b.memid > 0
GROUP BY f.name
ORDER BY member_usage DESC
LIMIT 5;
"""

pd.read_sql_query(query_12, engine)

,name,member_usage
0,Pool Table,783
1,Snooker Table,421
2,Massage Room 1,421
3,Table Tennis,385
4,Badminton Court,344


### Q13: Find the facilities usage by month, but not guests

In [20]:
query_13 = """
SELECT DISTINCT f.name, strftime('%m', starttime) AS month, COUNT(*) 
FROM Bookings AS b
JOIN Facilities AS f
ON f.facid = b.facid
JOIN Members AS m
ON m.memid = b.memid
WHERE b.memid > 0
GROUP BY f.name, month
ORDER BY f.name, month
LIMIT 5;
"""
pd.read_sql_query(query_13, engine)

,name,month,COUNT(*)
0,Badminton Court,07,51
1,Badminton Court,08,132
2,Badminton Court,09,161
3,Massage Room 1,07,77
4,Massage Room 1,08,153
